In [1]:
# Import open3d

import open3d as o3d
point_cloud_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/data_odometry_velodyne/dataset/sequences/21_onecar_pcd/velodyne"
pc_id = 380 
bb_id = 380- 361
# Load point cloud
# point_cloud = o3d.io.read_point_cloud(point_cloud_path+ "/" + "000423.pcd")
# point_cloud = o3d.io.read_point_cloud(point_cloud_path+ "/" + "000361.pcd")
point_cloud = o3d.io.read_point_cloud(point_cloud_path+ "/" + "000380.pcd")
print("Point cloud loaded")
# View point cloud
o3d.visualization.draw_geometries([point_cloud])



Point cloud loaded


In [2]:
# Load bounding box information file for all the frames
# Path: /home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/kiss-icp/python/results/21_onecar_odom/velodyne_poses_tum.txt

bb_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/OpenPCDet/tools/bb_pose.txt"
bb_poses = []
with open(bb_path, 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        # Convert to float
        line = [float(i) for i in line]
        bb_poses.append(line)
# print(bb_poses)

print("bb_poses", bb_poses[bb_id])

bb_poses [16.151415, 4.3119817, -0.84210455, 4.029848, 1.6493009, 1.5529932, -0.0371855]


In [3]:
import numpy as np
# Bounding Box Pose : Height, Width, Length, X, Y, Z, Yaw,
def compute_bounding_box_object(X, Y, Z, Length, Width, Height, Yaw):
    # Create the bounding box vertices in its local frame
    # vertices = np.array([
    #     [Length / 2, 0, Width / 2],
    #     [Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, Width / 2],
    #     [Length / 2, Height, Width / 2],
    #     [Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, Width / 2]
    # ])

    vertices = np.array([
        [Length / 2, Width / 2, -Height/2],
        [Length / 2, -Width / 2, -Height/2],
        [-Length / 2, -Width / 2, -Height/2],
        [-Length / 2, Width / 2, -Height/2],
        [Length / 2, Width / 2, Height/2],
        [Length / 2, -Width / 2, Height/2],
        [-Length / 2, -Width / 2, Height/2],
        [-Length / 2, Width / 2, Height/2]

    ])

    # Create the rotation matrix based on Yaw
    rotation_matrix = np.array([
        [np.cos(Yaw), 0, np.sin(Yaw)],
        [0, 1, 0],
        [-np.sin(Yaw), 0, np.cos(Yaw)]
    ])

    # Rotate and translate the bounding box to its final pose
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    translated_vertices = rotated_vertices + np.array([X, Y, Z])

    return o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(translated_vertices))

def compute_bounding_box_edges(X, Y, Z, Length, Width, Height,Yaw ):
    # Create the bounding box vertices in its local frame
    # vertices = np.array([
    #     [Length / 2, 0, Width / 2],
    #     [Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, Width / 2],
    #     [Length / 2, Height, Width / 2],
    #     [Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, Width / 2]
    # ])

    vertices = np.array([
        [Length / 2, Width / 2, -Height/2],
        [Length / 2, -Width / 2, -Height/2],
        [-Length / 2, -Width / 2, -Height/2],
        [-Length / 2, Width / 2, -Height/2],
        [Length / 2, Width / 2, Height/2],
        [Length / 2, -Width / 2, Height/2],
        [-Length / 2, -Width / 2, Height/2],
        [-Length / 2, Width / 2, Height/2]

    ])
    # Create the rotation matrix based on Yaw
    rotation_matrix = np.array([
        [np.cos(Yaw), 0, np.sin(Yaw)],
        [0, 1, 0],
        [-np.sin(Yaw), 0, np.cos(Yaw)]
    ])

    # Rotate and translate the bounding box to its final pose
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    translated_vertices = rotated_vertices + np.array([X, Y, Z])

        # Define the edges of the bounding box
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]
    # Create a Homogenous transformation matrix
    T = np.eye(4)
    T[:3, :3] = rotation_matrix
    T[:3, 3] = np.array([X, Y, Z])

    return o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(translated_vertices),
        lines=o3d.utility.Vector2iVector(edges)
    ), T
    # return o3d.utility.Vector3dVector(translated_vertices)

# Create a bbox object
bb_object = compute_bounding_box_object(*bb_poses[bb_id])
bb_edges, T = compute_bounding_box_edges(*bb_poses[bb_id])
print("bb_edges", bb_edges)
print("bb_object", type(bb_object))
o3d.visualization.draw_geometries([point_cloud, bb_edges])

# Visualize bb_edges
# o3d.visualization.draw_geometries([bb_object])

bb_edges LineSet with 12 lines.
bb_object <class 'open3d.cuda.pybind.geometry.OrientedBoundingBox'>


In [4]:
car_object = point_cloud.crop(bb_object)
# car_object_canonical = car_object

# Copy the point cloud to a new variable by just copying the data
import copy
car_object_canonical = copy.deepcopy(car_object)
car_object_canonical.transform(np.linalg.inv(T))
print(type(car_object))
# add coordinate frame
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
# Save the cropped point cloud to a file
o3d.io.write_point_cloud("cropped_car.pcd", car_object)
o3d.io.write_point_cloud("cropped_car_canonical.pcd", car_object_canonical)

o3d.visualization.draw_geometries([car_object, car_object_canonical, coordinate_frame])


<class 'open3d.cuda.pybind.geometry.PointCloud'>


In [5]:
# Read cropped_car.pcd and visualize it
car = o3d.io.read_point_cloud("cropped_car.pcd")
print("car", car)
# add coordinate frame
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
# visualize
vis = o3d.visualization.Visualizer()
o3d.visualization.draw_geometries([car, coordinate_frame])
# convert to numpy array
car_np = np.asarray(car.points)
print(car_np.shape)

car PointCloud with 411 points.
(411, 3)


In [6]:
# Run clustering on point cloud data
import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler



# Convert the point cloud to a NumPy array
points = np.asarray(car_object.points)

# Normalize the points (optional but recommended for DBSCAN)
scaler = StandardScaler()
normalized_points = scaler.fit_transform(points)

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.4, min_samples=10)  # Adjust eps and min_samples as per your data
labels = dbscan.fit_predict(normalized_points)

# Create a new point cloud with cluster labels as colors
colored_point_cloud = o3d.geometry.PointCloud()
colored_point_cloud.points = o3d.utility.Vector3dVector(points)
colored_point_cloud.colors = o3d.utility.Vector3dVector(np.random.random((len(points), 3)))

# Assign cluster labels to the colors
for i, label in enumerate(labels):
    if label == -1:  # Noise points will be shown in black
        colored_point_cloud.colors[i] = [0, 0, 0]
    else:
        # Assign random colors to clusters
        np.random.seed(label)
        colored_point_cloud.colors[i] = np.random.random(3)


# Visualize the clustered point cloud
o3d.visualization.draw_geometries([colored_point_cloud])


In [7]:
'''
# Multiply [[ 0.2394, -0.0139,  0.0310,  0.0379],
        [ 0.0198,  0.2364, -0.0466, -0.0317],
        [-0.0277,  0.0487,  0.2352, -0.1744],
        [ 0.0000,  0.0000,  0.0000,  1.0000]] and [0.93969262, -0.34202014, 0., 0.],
                            [0.34202014, 0.93969262, 0., 0.],
                            [0., 0., 1., 0.],
                            [0., 0., 0., 1.]]
'''
T1 = np.array([[ 0.2533,  0.0611,  0.0630,  0.0826],
        [-0.0536,  0.2601, -0.0367, -0.0036],
        [-0.0694,  0.0220,  0.2580, -0.0211],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

T2 = np.array([[0.93969262, -0.34202014, 0., 0.],
                                [0.34202014, 0.93969262, 0., 0.],
                                [0., 0., 1., 0.],
                                [0., 0., 0., 1.]])

T = np.matmul(T1, T2)

print(T)

[[ 0.25892157 -0.02921848  0.063       0.0826    ]
 [ 0.03859191  0.26274633 -0.0367     -0.0036    ]
 [-0.05769022  0.04440944  0.258      -0.0211    ]
 [ 0.          0.          0.          1.        ]]


In [8]:
from joint_opt import joint_optimization
scene_pcd, obj_pcd, car_points_after_correction_pcd, mesh_o3d, correction_transform = joint_optimization("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/P04_Instance-Completion-and-Motion-Estimation-with-Deep-Shape-Priors-for-Autonomous-Driving/DEEP_SDF/configs/config.yaml", car)

car PointCloud with 411 points.
car_points.shape (411, 3)
t_cam_obj initializing with I tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Object joint optimization: Iter 0, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 1, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 2, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 3, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 4, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 5, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 6, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 7, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 8, loss: 4.937500, sdf loss: 0.049375, 
Object joint optimization: Iter 9, loss: 4.937500, sdf loss: 0.049375, 
Reconstruction takes 0.650978 seconds
t_cam_obj adjustment needed tensor([[1., 0., 0., 

In [9]:
# visualize the mesh
o3d.visualization.draw_geometries([car, coordinate_frame, mesh_o3d, car_points_after_correction_pcd])
